###Association Rules - Apriori

#Author: Prof. Sandro Camargo <github.com/sandrocamargo>

Data Mining Course https://moodle.unipampa.edu.br/moodle/course/view.php?id=5213

This script uses the basic concepts of association rules.

In [1]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=5ae9e765f635f96e5fc89cd30b14291f05b2e93e53014356e61b82f1a2f50713
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from apyori import apriori

In [12]:
# Import Data from CSV file
dataset = pd.read_csv('/content/groceries - groceries.csv',sep=',')

In [13]:
# View the dataset
dataset.head(10)

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,whole milk,butter,yogurt,rice,abrasive cleaner,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,rolls/buns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,potted plants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,whole milk,cereals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Apriori algorithm takes the list of items that were bought together as input. Hence, we need to get each row
# as list (except 1st column and 'NAN' in the columns).
# Create a list of trasactions
transactions = []

# Add all the items from each row in one list( Neglect the 1st columns where all the items are in number (0-9))
for i in range(0, 9835):
    transactions.append([str(dataset.values[i,u]) for u in range(1, 33)])

In [67]:
# Training the Apriori Algorithms
rules = apriori(transactions, min_support=0.02, min_confidence=0.2, min_lift=1, min_length = 2)

# Min_support  = 3(3 times a day) * 7 (7 days a week) / 9835 = 0.0022
# Min_confidence = set it lower to get more relations between products (weak relations), if we set it high then
# we might miss some. I have selected confidence of 0.20
# Min_lift = In order to get some relevant rules, I am setting min_lift to 3.

In [68]:
# Store rules in result variable
results = list(rules)

# See the items that were bought together with their support
results_list = []
for i in range(0, len(results)):
    results_list.append('RULE:' + str(results[i][0]) + '\nSUP:' + str(results[i][1]) + '\nCONF:' + str(results[i][2]) + '\n\n')

In [69]:
print("The list of rules generated by the Apriori algorithm are:")
for i in range(0, len(results_list)):
    print(results_list[i])

The list of rules generated by the Apriori algorithm are:
RULE:frozenset({'nan'})
SUP:0.9998983223182512
CONF:[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=0.9998983223182512, lift=1.0)]


RULE:frozenset({'whole milk'})
SUP:0.25551601423487547
CONF:[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'whole milk'}), confidence=0.25551601423487547, lift=1.0)]


RULE:frozenset({'nan', 'UHT-milk'})
SUP:0.03345195729537367
CONF:[OrderedStatistic(items_base=frozenset({'UHT-milk'}), items_add=frozenset({'nan'}), confidence=1.0, lift=1.000101688021151)]


RULE:frozenset({'whole milk', 'beef'})
SUP:0.02125063548551093
CONF:[OrderedStatistic(items_base=frozenset({'beef'}), items_add=frozenset({'whole milk'}), confidence=0.4050387596899225, lift=1.5851795469758803)]


RULE:frozenset({'nan', 'berries'})
SUP:0.033248601931875954
CONF:[OrderedStatistic(items_base=frozenset({'berries'}), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0001016880211